In [1]:
# This script goes along the blog post
# "Building powerful image classification models using very little data"
# from blog.keras.io.
# Modifications by Yannis Georgas 5-Jan-2019
# data for training and validation sets was scrapped and cleaned from instagram, flickr and other websites

import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 250, 250

top_model_weights_path = 'bottleneck_fc_model_smax.h5'
train_data_dir = 'data/Train'
validation_data_dir = 'data/Validation'
nb_train_samples = 3360  # that is 3 times 1120 
nb_validation_samples = 1200   # 1200/ 3 = 400
epochs = 50
batch_size = 16 # 3360 / 16 = 210 (clean integer) and for validation 1200/16 = 75 (clean integer) 
#if its not clean integer it will blow up because its not able to divide for mini batch calculation


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save('bottleneck_features_train_smax.npy',bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save('bottleneck_features_validation_smax.npy',bottleneck_features_validation)


def train_top_model():
        
    k = 3 # number of classes but actually 
    
    train_data = np.load('bottleneck_features_train_smax.npy')
    train_labels = np.array([0] * (nb_train_samples // k) + 
                            [1] * (nb_train_samples // k) + 
                            [2] * (nb_train_samples // k)) 

    validation_data = np.load('bottleneck_features_validation_smax.npy')
    validation_labels = np.array([0] * (nb_validation_samples // k) + 
                                 [1] * (nb_validation_samples // k) + 
                                 [2] * (nb_validation_samples // k))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(k, activation='softmax'))  # 3 for tshirt, hoodie, blouse - softmax (Dense 1 for binary class - sigmoid)

    model.compile(optimizer='rmsprop',
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])   #binary_crossentropy for sigmoid

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    
    
save_bottlebeck_features()
train_top_model()

Using TensorFlow backend.


Found 6720 images belonging to 6 classes.
Found 2400 images belonging to 6 classes.
Train on 6720 samples, validate on 2400 samples
Epoch 1/50
6720/6720 [==============================] - 31s 5ms/step - loss: 1.4023 - acc: 0.7060 - val_loss: 0.6040 - val_acc: 0.7987
Epoch 2/50
6720/6720 [==============================] - 31s 5ms/step - loss: 0.5846 - acc: 0.8060 - val_loss: 0.6069 - val_acc: 0.8242
Epoch 3/50
6720/6720 [==============================] - 31s 5ms/step - loss: 0.5022 - acc: 0.8384 - val_loss: 0.5849 - val_acc: 0.8408
Epoch 4/50
6720/6720 [==============================] - 31s 5ms/step - loss: 0.4767 - acc: 0.8496 - val_loss: 0.5963 - val_acc: 0.8346
Epoch 5/50
6720/6720 [==============================] - 31s 5ms/step - loss: 0.4157 - acc: 0.8674 - val_loss: 0.9266 - val_acc: 0.8146
Epoch 6/50
6720/6720 [==============================] - 31s 5ms/step - loss: 0.4069 - acc: 0.8738 - val_loss: 0.8594 - val_acc: 0.8492
Epoch 7/50
6720/6720 [==============================] - 31

NOTE    
    If your targets are one-hot encoded, use categorical_crossentropy.
        Examples of one-hot encodings:
            [1,0,0]
            [0,1,0]
            [0,0,1]
    But if your targets are integers, use sparse_categorical_crossentropy.
        Examples of integer encodings (for the sake of completion):
            1
            2
            3
